# VME Charge Particle Tutorial

This is a simple tutorial based on some code I put together in the control room during the WCTE run. That code was originally inspired by Arturo’s C++ version. Hopefully, this version gives you a good starting point to create your own.

It’s definitely not a full analysis code! Just something to help you get familiar with the VME data. I hope you have fun playing around with it.

If you end up enjoying it, the WCTE beam group has lots of things to do, and we’d be love to have your help.
 
Bruno Ferrazzi, Aug/2025.

# RAW DATA

First things first, load the libraries:

In [ ]:
import uproot
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

Add the full path to your file:

In [ ]:
file_path    = "/home/bferrazzi/Desktop/notebooks/data/WCTE_offline_R1308S0_VME1406.root"
tree_name    = "WCTEReadoutWindows"        # This will not change

Below is a simple function to plot the TDC and QDC straight from the file for a given a channel id

In [ ]:
def plot_time_charge_distributions(
    root_path: str,
    channel_id: int,
    tree_name: str = "WCTEReadoutWindows",
    bins: int = 500  # Change the bin number if needed
):

    # open file with uproot and grab arrays
    with uproot.open(root_path) as f:
        tree = f[tree_name]
        arrs = tree.arrays(
            ["beamline_pmt_tdc_times",
             "beamline_pmt_tdc_ids",
             "beamline_pmt_qdc_charges",
             "beamline_pmt_qdc_ids"],
            library="np"
        )

    # make sure they are flat numpy arrays
    tdc_times = np.concatenate(arrs["beamline_pmt_tdc_times"])
    tdc_ids   = np.concatenate(arrs["beamline_pmt_tdc_ids"])
    qdc_chg   = np.concatenate(arrs["beamline_pmt_qdc_charges"])
    qdc_ids   = np.concatenate(arrs["beamline_pmt_qdc_ids"])

    # select only entries matching our channel id
    times_for_id   = tdc_times[tdc_ids == channel_id]
    charges_for_id = qdc_chg[  qdc_ids == channel_id]

    # plot TDC times
    plt.figure()
    plt.hist(times_for_id, bins=bins)
    plt.xlabel("TDC time (ns)")
    plt.ylabel("Entries")
    plt.yscale("log")
    plt.title(f"TDC Time Distribution — Channel {channel_id}")
    plt.show()

    # plot QDC charges
    plt.figure()
    plt.hist(charges_for_id, bins=bins, color='red')
    plt.yscale("log")
    plt.xlabel("QDC charge (arb. units)")
    plt.ylabel("Entries")
    plt.title(f"QDC Charge Distribution — Channel {channel_id}")
    plt.show()

Exercise 1: Getting familiar with TDC and QDC shape.

The channel map can be found here: https://wcte.hyperk.ca/operations/shift-instructions-and-manuals/manuals/vme-data-format

You need to able to answer those questions:

i) What is the second peak (~200) on T0 and T1 TDC channels?

ii) Why T0 and T1 don't have QDC pedestal?

iii) Can you identify p.e. structure on ACTs QDC?

iV) Why the TDC peaks in general looks so wide is it bad timing resolution?

V) Why the TOF-detector QDC is so different?

Vi) What value should be the QDC cut on T4? (You will need this value to proceed)

In [ ]:
plot_time_charge_distributions(file_path, channel_id=12) # Change the channel id

# DETECTION TECHNIQUES

Make some definitions here:

In [ ]:
reference_ids = (31, 46)          # (TDC ref for IDs <31, ref1 for IDs >31)
t0_group     = [0, 1, 2, 3]       # must all be present
t1_group     = [4, 5, 6, 7]       # must all be present
t4_group     = [42, 43]           # must all be present
t4_qdc_cut   = 000                # Only hits above this value
act_eveto_group = [12, 13, 14, 15, 16, 17]   # ACT-eveto channels
act_tagger_group = [18, 19, 20, 21, 22, 23]
hc_group = [9, 10]
hc_charge_cut = 150 # Only hits below this value

Let's open the file and get the branches

In [ ]:
# Open the file and grab the tree
f    = uproot.open(file_path)
tree = f[tree_name]

# Load all four branches into NumPy arrays
branches = [
    "beamline_pmt_tdc_times",
    "beamline_pmt_tdc_ids",
    "beamline_pmt_qdc_charges",
    "beamline_pmt_qdc_ids",
]
data = tree.arrays(branches, library="np")

Let's make our first TOF-measurement using T0 and T1

In [ ]:
# ——— prepare lists ———
t0_avgs  = []
t1_avgs  = []
tof_vals = []

for evt_idx in range(len(data[branches[0]])):
    tdc_times = data["beamline_pmt_tdc_times"][evt_idx]
    tdc_ids   = data["beamline_pmt_tdc_ids"][evt_idx]

    # reference-time subtraction & first-hit only
    mask0 = (tdc_ids == reference_ids[0])
    mask1 = (tdc_ids == reference_ids[1])
    if not mask0.any() or not mask1.any():
        continue
    ref0 = tdc_times[mask0][0]
    ref1 = tdc_times[mask1][0]

    corrected = {}
    for ch, t in zip(tdc_ids, tdc_times):
        if ch in reference_ids or ch in corrected:
            continue
        corrected[ch] = t - (ref0 if ch < reference_ids[0] else ref1)
    
    # require all channels on T0/T1 before computing averages
    if not all(ch in corrected for ch in t0_group+t1_group):
        continue

    # compute the averages
    t0 = np.mean([corrected[ch] for ch in t0_group])
    t1 = np.mean([corrected[ch] for ch in t1_group])
    t0_avgs.append(t0)
    t1_avgs.append(t1)
    tof_vals.append(t1 - t0)

# ——— plot histograms ———
plt.figure()
plt.hist(t0_avgs, bins=50)
plt.xlabel("⟨T0⟩ (ns)")
plt.ylabel("Counts")
plt.title("T0 average time distribution")
plt.show()

plt.figure()
plt.hist(t1_avgs, bins=50)
plt.xlabel("⟨T1⟩ (ns)")
plt.ylabel("Counts")
plt.title("T1 average time distribution")
plt.show()

plt.figure()
plt.hist(tof_vals, bins=1000)
plt.xlabel("T1–T0 (ns)")
plt.ylabel("Counts")
plt.xlim(10,20)
# plt.yscale("log")
plt.title("TOF (T0 minus T1) distribution")
plt.show()


Exercise 2: TOF

i) Each peak represents a particle, which ones?

ii) Which particles is the fastest ?

iii) Given the fastest particle and the T0 to T1 distance, what is the predicted TOF time?

iii-i)Distances can be found here: https://wcte.hyperk.ca/operations/shift-instructions-and-manuals/manuals/beam-alignment/2025-position-measurements/measured-positions

iii-i) You will need the particle mass, google it.

Run the cell below. It will ask for the particle mass, momentum and detector distance and it will give you the TOF.

In [ ]:
# Constants
c = 299_792_458  # speed of light in m/s

# Some inputs
mass_mev = float(input("Enter particle mass [MeV/c²]: "))
momentum_mev = float(input("Enter particle momentum [MeV/c]: "))
distance_m = float(input("Enter distance [m]: "))

# Compute total energy and beta
mass = mass_mev
p = momentum_mev
E = np.sqrt(p**2 + mass**2)
beta = p / E  # v/c

# Time-of-flight calculation
tof_seconds = distance_m / (beta * c)
tof_ns = tof_seconds * 1e9

print(f"\nExpected TOF: {tof_ns:.2f} ns")

iV) Does the peak match the prediction? If not, why?

V) If you want to measure the momentum what would be your plan?

Let's plot the sum of ACTs used for electron veto

In [ ]:
act_eveto_sums = []
n_events   = len(data[branches[0]])

for evt_idx in range(n_events):
    qdc_charges = data["beamline_pmt_qdc_charges"][evt_idx]
    qdc_ids     = data["beamline_pmt_qdc_ids"][evt_idx]

    qdc_dict = {}
    for ch, q in zip(qdc_ids, qdc_charges):
        if ch not in qdc_dict:
            qdc_dict[ch] = q

    # sum over your all eveto group (missing channels contribute 0)
    s = sum(qdc_dict.get(ch, 0) for ch in act_eveto_group)
    act_eveto_sums.append(s)

# ——— PLOT THE HISTOGRAM ———
plt.figure()
plt.hist(act_eveto_sums, bins=100)
plt.xlabel("ACT-eveto total QDC charge")
plt.ylabel("Counts")
plt.yscale("log")
plt.title("Histogram of summed ACT-eveto charges")
plt.show()

Exercise 3: E-veto

i) Why we sum the charges of different act boxes?

ii) How many peaks can you identify and which particles they represent?

iii) Where should be the electron cut position? (You will need this value to continue)

In [ ]:
act_eveto_cut = 0000   # ignore events if sum is above this value

iV) What are the index of the ACTs 0,1,2 for this run?

V) What are the momentum therehold for light production in those ACTs? How this impacts the veto?

V-i) Run the cell below to calculate the threshold given the ACT index and particle mass

VME log:https://docs.google.com/spreadsheets/d/1_gLpa4sqnzdQ0KSNIQISDQMgOH_hka0FYeIbLpOANIQ/edit?gid=1045855780#gid=1045855780

In [ ]:
n = float(input("Enter index of refraction (n > 1): "))
mass_mev = float(input("Enter particle mass [MeV/c²]: "))

if n <= 1:
    print("No Cherenkov radiation: index must be > 1")
else:
    # Compute beta threshold
    beta_thr = 1 / n
    gamma_thr = 1 / np.sqrt(1 - beta_thr**2)

    # Compute momentum threshold
    p_thr = mass_mev * beta_thr * gamma_thr  # in MeV/c

    print(f"\nCherenkov momentum threshold: {p_thr:.2f} MeV/c")

Let's plot the ACTs used for pi/mu separation

In [ ]:
act_tagger_sums = []
n_events   = len(data[branches[0]])

for evt_idx in range(n_events):
    qdc_charges = data["beamline_pmt_qdc_charges"][evt_idx]
    qdc_ids     = data["beamline_pmt_qdc_ids"][evt_idx]

    # collect first-hit charges into a dict (as before)
    qdc_dict = {}
    for ch, q in zip(qdc_ids, qdc_charges):
        if ch not in qdc_dict:
            qdc_dict[ch] = q

    # sum over your eveto group (missing channels contribute 0)
    s = sum(qdc_dict.get(ch, 0) for ch in act_tagger_group)
    act_tagger_sums.append(s)

# ——— PLOT THE HISTOGRAM ———
plt.figure()
plt.hist(act_tagger_sums, bins=100)
plt.xlabel("ACT-tagger total QDC charge")
plt.ylabel("Counts")
plt.yscale("log")
plt.title("Histogram of summed ACT-tagger charges")
plt.show()

Exercise 4: Pi/Mu

i) How many peaks can you identify and which particles they represent?

ii) There seems to be a charge saturation why?

iii) If you want to separate pions and muons where would cut?

iV) What are the index for ACTs 3,4,5 for this run?

V) Go back to the cell with the threshold calculation and check what signals it should be expected here?

# DATA CUT

This is simple event-by-event cut system. It is not the most efficient way but it is easier to understand and it is closer to the C++ approach. Look for vectorization, slicing and chunking in python to see how this can be improved.

In [ ]:
# This list gives you the ROOT entries that survived the cut
selected_events = []

# timer
start_time = time.time()

# Loop over ROOT events
for evt_idx in range(len(data[branches[0]])):
    tdc_times   = data["beamline_pmt_tdc_times"][evt_idx]
    tdc_ids     = data["beamline_pmt_tdc_ids"][evt_idx]
    qdc_charges = data["beamline_pmt_qdc_charges"][evt_idx]
    qdc_ids     = data["beamline_pmt_qdc_ids"][evt_idx]

    # Correct the TDC time and only keep first hits
    mask0 = (tdc_ids == reference_ids[0])
    mask1 = (tdc_ids == reference_ids[1])
    if not mask0.any() or not mask1.any():
        continue
    ref0_time = tdc_times[mask0][0]
    ref1_time = tdc_times[mask1][0]

    corrected = {}
    for ch, t in zip(tdc_ids, tdc_times):
        if ch in reference_ids or ch in corrected:
            continue
        corrected[ch] = t - (ref0_time if ch < reference_ids[0] else ref1_time)

    qdc_dict = {}
    for ch, q in zip(qdc_ids, qdc_charges):
        if ch not in qdc_dict:
            qdc_dict[ch] = q

    #------Electron veto CUT------#
    eveto_sum = sum(qdc_dict.get(ch, 0) for ch in act_eveto_group)
    if eveto_sum > act_eveto_cut:
        continue  # ignore events

    #------T4 CUT------#
    if not all(ch in qdc_dict for ch in t4_group): # Requires all channels
        continue
    if not all(qdc_dict[ch] > t4_qdc_cut for ch in t4_group): # needs to be above
        continue
        
    #------HC CUT------#
    if any(qdc_dict.get(ch, 0) >= hc_charge_cut for ch in hc_group):
        continue # if either HC channel fired with charge ≥ threshold, skip event
    
    #------T0 / T1 CUT------#
    if all(ch in corrected for ch in t0_group) and all(ch in corrected for ch in t1_group):
        selected_events.append(evt_idx)

# timer
end_time = time.time()
elapsed = end_time - start_time

n_events = len(data[branches[0]])
print(f"Processed {n_events} events in {elapsed:.2f} s "
      f"({elapsed/n_events*1e3:.3f} ms/event).")
print(f"Events passing first selection: {len(selected_events)}")
print("First 10 passing event indices:", selected_events[:10])

Exercise 5: T4 Cut

i) With T4 cut enabled on the code, make the plots below.

Let's plot the TOF again:

In [ ]:
# Prepare output lists
t0_avgs, t1_avgs, tof_vals = [], [], []

for evt_idx in selected_events:
    tdc_times   = data["beamline_pmt_tdc_times"][evt_idx]
    tdc_ids     = data["beamline_pmt_tdc_ids"][evt_idx]

    # get reference times again
    mask0 = (tdc_ids == reference_ids[0])
    mask1 = (tdc_ids == reference_ids[1])
    ref0 = tdc_times[mask0][0]
    ref1 = tdc_times[mask1][0]

    # apply our “subtract and first‐hit” correction
    corrected = {}
    for ch, t in zip(tdc_ids, tdc_times):
        if ch in reference_ids or ch in corrected:
            continue
        corrected[ch] = t - (ref0 if ch < reference_ids[0] else ref1)

    # now compute T0/T1 averages and TOF
    t0 = np.mean([corrected[ch] for ch in t0_group])
    t1 = np.mean([corrected[ch] for ch in t1_group])
    t0_avgs.append(t0)
    t1_avgs.append(t1)
    tof_vals.append(t1 - t0)

plt.figure()
plt.hist(tof_vals, bins=500)
plt.xlabel("T1–T0 (ns)")
plt.ylabel("Counts")
plt.xlim(10,20)
plt.title("TOF (T1 minus T0) distribution")
plt.show()


ii) What changed compared to TOF plot that we did before?

iii) Which particles survived? Is the cut 100% efficient?

Let's plot the act charge versus TOF:

In [ ]:
tagger_sel = []

for evt_idx in selected_events:
    qdc_ids     = data["beamline_pmt_qdc_ids"][evt_idx]
    qdc_charges = data["beamline_pmt_qdc_charges"][evt_idx]

    # build first‐hit map
    qdc_dict = {}
    for ch, q in zip(qdc_ids, qdc_charges):
        if ch not in qdc_dict:
            qdc_dict[ch] = q

    # sum charges on the tagger group
    tagger_sel.append(sum(qdc_dict.get(ch, 0) for ch in act_tagger_group))

# ——— Plot 2D histogram of TOF vs. tagger charge ———
plt.figure(figsize=(8,6))
hb = plt.hist2d(tof_vals, tagger_sel,norm=LogNorm(), bins=(200,200))
plt.colorbar(hb[3], label="Counts")
plt.xlabel("T1 – T0 (ns)")
plt.ylabel("ACT‐tagger total QDC charge")
plt.title("2D Histogram: Tagger Charge vs. TOF")
plt.xlim(10,20)
plt.tight_layout()
plt.show()


iV) Which particle can you identify?

V) Go back to the cutting code and comment it out the T4 cut. Run the plots again, what changed?

V-i) This was a big problem in 2023. Alie named them "weird electrons", can you explain why she chose this name? 

V-ii) What solved the "weird electrons" problems? why?

Vi) Can you estimate you cut efficiency by event couting on the cutting code?

Viii) Implement a cut on the Muon Tagger

iX) Implement a cut on the TOF-detector

# THE END, Thanks